In [1]:
from astropy.io import fits
import sys
import pandas as pd
from astropy.table import unique, Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc, sky_match
from numcosmo_py.external.pyssc import pyssc as PySSC
from numcosmo_py.helper import npa_to_seq
import timeit
Ncm.cfg_init()

from getdist.mcsamples import  MCSamples
from getdist import plots
import getdist

# Cosmological Model

In [2]:
# Taking some important information about the fiducial cosmological model and survey region
sky_area = 439.790

#cosmological model
cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("Omegak",0.0)
cosmo.param_set_by_name("H0", 71)
cosmo.param_set_by_name("Omegab", 0.0448)
cosmo.param_set_by_name("Omegac", 0.22)  # 0.2603

cosmo.param_set_by_name("w", -1.0)  # -1.0

prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA",  3.05991748699321)
prim.props.n_SA = 0.963
reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)

#matter power spectrum
tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()
psf.prepare(cosmo)
print(cosmo.sigma8(psf))

dist = Nc.Distance.new(2.0)
dist.prepare(cosmo)

0.8000000000000014


# Cluster Mass-Richness and photoz model 

In [3]:
cluster_m = Nc.ClusterMassAscaso(lnRichness_min = 0.0 ,lnRichness_max = 6.0)
cluster_m.param_set_by_name("mup0", 4.11945317984568)
cluster_m.param_set_by_name("mup1", 1.11236863168254)
cluster_m.param_set_by_name("mup2", 0.299464198653602)
cluster_m.param_set_by_name("sigmap0", 0.471246505057074)
cluster_m.param_set_by_name("sigmap1", -0.0591674971985774)
cluster_m.param_set_by_name("sigmap2", -0.109077938873429)
cluster_m.param_set_by_name("cut", np.log(1.0))
cluster_m.set_enable_rejection(True)

cluster_m.param_set_desc("mup0",{ "lower-bound": 3.0,"upper-bound": 5.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 4.11945317984568})
cluster_m.param_set_desc("mup1",{ "lower-bound": 0.0,"upper-bound": 3.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 1.11236863168254})
cluster_m.param_set_desc("mup2",{ "lower-bound": -1.0,"upper-bound": 1.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 0.299464198653602})
cluster_m.param_set_desc("sigmap0",{ "lower-bound": 0.1,"upper-bound": 2.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 0.471246505057074})
cluster_m.param_set_desc("sigmap1",{ "lower-bound": -0.1,"upper-bound": -0.00001,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": -0.0591674971985774})
cluster_m.param_set_desc("sigmap2",{ "lower-bound": -0.3,"upper-bound": 0.1,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value":-0.109077938873429})


cut = np.log(20.0)
cluster_m_truncated = Nc.ClusterMassAscaso(lnRichness_min = cut ,lnRichness_max = 6.0)
cluster_m_truncated.param_set_by_name("mup0", 4.11945317984568)
cluster_m_truncated.param_set_by_name("mup1", 1.11236863168254)
cluster_m_truncated.param_set_by_name("mup2", 0.299464198653602)
cluster_m_truncated.param_set_by_name("sigmap0", 0.471246505057074)
cluster_m_truncated.param_set_by_name("sigmap1", -0.0591674971985774)
cluster_m_truncated.param_set_by_name("sigmap2", -0.109077938873429)
cluster_m_truncated.param_set_by_name("cut", cut)

cluster_m_truncated.param_set_desc("mup0",{ "lower-bound": 3.0,"upper-bound": 5.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 4.11945317984568})
cluster_m_truncated.param_set_desc("mup1",{ "lower-bound": 0.0,"upper-bound": 3.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 1.11236863168254})
cluster_m_truncated.param_set_desc("mup2",{ "lower-bound": -1.0,"upper-bound": 1.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 0.299464198653602})
cluster_m_truncated.param_set_desc("sigmap0",{ "lower-bound": 0.2,"upper-bound": 2.0,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": 0.471246505057074})
cluster_m_truncated.param_set_desc("sigmap1",{ "lower-bound": -0.1,"upper-bound": -0.00001,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value": -0.0591674971985774})
cluster_m_truncated.param_set_desc("sigmap2",{ "lower-bound": -0.3,"upper-bound": 0.1,"scale": 1.0e-2,"abstol": 1.0e-50,"fit": True,"value":-0.109077938873429})

cluster_z = Nc.ClusterRedshiftNodist(z_min = 0.0 ,z_max = 1.1)

# Cluster abundance model

In [4]:
mulf = Nc.MultiplicityFuncDespali.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.VIRIAL)

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.prepare(cosmo)
hbias = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf, hbias)
cad.set_area(sky_area * (np.pi / 180) ** 2)
cad.prepare(cosmo , cluster_z , cluster_m)
print(cad.n(cosmo ,cluster_z , cluster_m))

183710.07160070643


In [5]:
rng = Ncm.RNG.seeded_new(None, 13)
mset = Ncm.MSet.new_array([cluster_m, cosmo, cluster_z])
mset.prepare_fparam_map()

ncdata = Nc.DataClusterNCount.new(cad, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ncdata.init_from_sampling(mset, sky_area * (np.pi / 180) ** 2  , rng)

lnM_obs_data = np.array([ncdata.get_lnM_obs().get_col(0).get(i) for i in range(ncdata.get_lnM_obs().get_col(0).len())])
lnM_true     = np.array([ncdata.get_lnM_true().get(i) for i in range(ncdata.get_lnM_obs().get_col(0).len())])
z_obs_data    = np.array([ncdata.get_z_obs().get_col(0).get(i) for i in range(ncdata.get_z_obs().get_col(0).len())])
z_true   = np.array([ncdata.get_z_true().get(i) for i in range(ncdata.get_z_obs().get_col(0).len())])

lnM_v = Ncm.Vector.new_array(npa_to_seq(lnM_true))
z_v = Ncm.Vector.new_array(npa_to_seq(z_true))
rich_v = Ncm.Vector.new_array(npa_to_seq(lnM_obs_data))

dmr = Nc.DataClusterMassRich.new()
dmr.set_data(lnM_v, z_v, rich_v)

mset_truncated = Ncm.MSet.new_array([cluster_m_truncated, cosmo, cluster_z])
dmr.resample(mset , rng)
print(dmr.peek_lnR().len())

151861


In [6]:
#dmr.resample(mset_truncated , rng)
#print(dmr.peek_lnR().len())

4494


/tmp/ipykernel_1883019/1463760110.py:1: Warning: g_object_unref: assertion 'G_IS_OBJECT (object)' failed
  dmr.resample(mset_truncated , rng)


# Fitting

## Fit

In [ ]:
dset = Ncm.Dataset.new()
dset.append_data(dmr)
lh = Ncm.Likelihood.new(dset)

fit = Ncm.Fit.factory(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.log_info()
fit.run(Ncm.FitRunMsgs.SIMPLE)
fit.obs_fisher()
fit.log_covar()

## MC

In [ ]:
Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

fitmc = Ncm.FitMC.new(fit, Ncm.FitMCResampleType.FROM_MODEL, Ncm.FitRunMsgs.SIMPLE)
fitmc.set_nthreads(2)
fitmc.set_data_file("mc_cut.fits")

fitmc.start_run()
fitmc.run(2000)
fitmc.end_run()
fitmc.mean_covar()
fit.log_covar()

mcat = Ncm.MSetCatalog.new_from_file_ro("mc_cut.fits", 0)
mcat.log_full_covar()
